In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
import IPython.display as display
from pathlib import Path
from sklearn.model_selection import train_test_split
from  soundfile import SoundFile
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool1D, BatchNormalization,Conv1D
import tensorflow as tf
from keras import Model
# from keras.applications import ResNet50
from keras.layers import AveragePooling2D,Lambda,ReLU,MultiHeadAttention,Embedding, LeakyReLU
from keras.optimizers import RMSprop,Adam
from tqdm.notebook import tqdm
from scipy.stats import skew
import joblib
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from math import floor

In [2]:
path = '../input/birdclef-2021/'
os.listdir(path)

['train_soundscape_labels.csv',
 'sample_submission.csv',
 'train_soundscapes',
 'train_short_audio',
 'test.csv',
 'train_metadata.csv',
 'test_soundscapes']

In [3]:
def read_ogg_file(path, file):
    """ Read ogg audio file and return numpay array and samplerate"""
    
    data, samplerate = sf.read(path+file)
    return data, samplerate


def plot_audio_file(data, samplerate):
    """ Plot the audio data"""
    
    sr = samplerate
    fig = plt.figure(figsize=(10,10))
    x = range(len(data))
    y = data
    plt.plot(x, y)
    plt.plot(x, y, color='red')
    plt.legend(loc='upper center')
    plt.grid()
    
    
def mel_spectrogram(data, samplerate):
    """ Plot spectrogram with mel scaling """
    
    sr = samplerate
    spectrogram = librosa.feature.melspectrogram(data, sr=sr,n_fft=int(0.025*32000),hop_length=715,n_mels=224)
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    return log_spectrogram
#     librosa.display.specshow(log_spectrogram, sr=sr, x_axis='time', y_axis='mel')

In [4]:
train_labels = pd.read_csv(path+'train_soundscape_labels.csv')
train_meta = pd.read_csv(path+'train_metadata.csv')
test_data = pd.read_csv(path+'test.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [5]:
print('Number train label samples:', len(train_labels))
print('Number train meta samples:', len(train_meta))
print('Number train short folder:', len(os.listdir(path+'train_short_audio')))
print('Number train audios:', len(os.listdir(path+'train_soundscapes')))
print('Number test samples:', len(test_data))

Number train label samples: 2400
Number train meta samples: 62874
Number train short folder: 397
Number train audios: 20
Number test samples: 3


In [6]:
def get_audio_info(filepath):
    """Get some properties from  an audio file"""
    with SoundFile(filepath) as f:
        sr = f.samplerate
        frames = f.frames
        duration = float(frames)/sr
    return {"frames": frames, "sr": sr, "duration": duration}

In [7]:
SR = 32_000
DURATION = 5
SEED = 666

DATA_ROOT = Path("../input/birdclef-2021")
TRAIN_AUDIO_ROOT = Path("../input/birdclef-2021/train_short_audio")
TRAIN_AUDIO_IMAGES_SAVE_ROOT = Path("audio_images") 
TRAIN_AUDIO_IMAGES_SAVE_ROOT.mkdir(exist_ok=True, parents=True)

In [8]:
def get_audio_info(filepath):
    """Get some properties from  an audio file"""
    with SoundFile(filepath) as f:
        sr = f.samplerate
        frames = f.frames
        duration = float(frames)/sr
    return {"frames": frames, "sr": sr, "duration": duration}

In [9]:
# list_IDs_train, list_IDs_val = train_test_split(range(len(train_data)), test_size=0.2, random_state=2021)
# list_IDs_test = list(samp_subm.index)

In [10]:
def make_df(n_splits=5, seed=SEED, nrows=None):
    
    df = pd.read_csv(DATA_ROOT/"train_metadata.csv", nrows=nrows)

    LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df["primary_label"].unique()))}
    
    df["label_id"] = df["primary_label"].map(LABEL_IDS)

    df["filepath"] = [str(TRAIN_AUDIO_ROOT/primary_label/filename) for primary_label,filename in zip(df.primary_label, df.filename) ]
    
    df['val']=0
    list_IDs_train, list_IDs_val = train_test_split(range(len(df)), test_size=0.2, random_state=2021,stratify=df['primary_label'])
    

    pool = joblib.Parallel(4)
    mapper = joblib.delayed(get_audio_info)
    tasks = [mapper(filepath) for filepath in df.filepath]

    df = pd.concat([df, pd.DataFrame(pool(tqdm(tasks)))], axis=1, sort=False)
    df.loc[list_IDs_val,'val']=1

    return LABEL_IDS, df

In [11]:
LABEL_IDS, train_meta = make_df(nrows=None)



  0%|          | 0/62874 [00:00<?, ?it/s]

In [12]:
train_meta.to_csv('train_metadata_enriched.csv',index=None)

In [13]:
with open('label_ids.pickle','wb') as f:
    pickle.dump(LABEL_IDS,f)

In [14]:
train_meta_sorted=train_meta.sort_values(by='duration',ascending=False)

In [15]:
AUDIO_LENGTH=int(SR*DURATION)

In [16]:
STEP=int(DURATION*0.666*SR)

In [17]:
def MelspecToImage(features):
    eps=1e-6
    mean = features.mean()
    std=features.std()
    features=(features-mean)/(std+eps)
    _min, _max = features.min(), features.max()
    if (_max - _min) > eps:
        V=np.clip(features, _min, _max)
        V=255 * (V - _min) / (_max - _min)
        V=V.astype(np.uint8)
    else:
        V = np.zeros_like(features, dtype=np.uint8)
    rgb_image=np.repeat(V[...,np.newaxis],3,axis=-1)
    return rgb_image


In [18]:
def crop_or_pad(y, length, is_train=True, start=None):
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length - len(y))])
        
        n_repeats = length // len(y)
        epsilon = length % len(y)
        
        y = np.concatenate([y]*n_repeats + [y[:epsilon]])
        
    elif len(y) > length:
        if not is_train:
            start = start or 0
        else:
            start = start or np.random.randint(len(y) - length)

        y = y[start:start + length]

    return y


In [19]:
def signal_score(audios):
    score=[]
    for aud in audios:
        q=np.quantile(aud,[0.2,0.8])
        # aud=aud-np.mean(aud)/np.std(aud)
        audio_clipped=np.clip(aud,q[0],q[1])
        scr=np.abs(np.quantile(aud,0.9)/np.mean(audio_clipped))*np.abs(skew(aud))
        score.append(scr)
    return score

In [20]:
if not os.path.exists('audio_images/train'):
    os.mkdir('audio_images/train')

if not os.path.exists('audio_images/val'):
    os.mkdir('audio_images/val')


In [21]:
for row in tqdm(train_meta.itertuples(False)):
    if row.val==0:
        if not os.path.exists("audio_images/train/{}".format(row.primary_label)):
            os.mkdir("audio_images/train/{}".format(row.primary_label))


0it [00:00, ?it/s]

In [22]:
for row in tqdm(train_meta.itertuples(False)):
    if row.val==1:
        if not os.path.exists("audio_images/val/{}".format(row.primary_label)):
            os.mkdir("audio_images/val/{}".format(row.primary_label))


0it [00:00, ?it/s]

In [23]:
def generate_and_save_image(row):
    file = row.filepath
    audio_file, audio_sr = sf.read(file)
    audio_file = audio_file[:int(AUDIO_LENGTH)]
    audio_file, audio_sr = sf.read(file)
    audios = [audio_file[i:i+AUDIO_LENGTH] for i in range(0, max(1, len(audio_file) - AUDIO_LENGTH + 1), STEP)]
    audios[-1] = crop_or_pad(audios[-1] , length=AUDIO_LENGTH)
    scores=signal_score(audios)
    if row.duration >=20:
        sorted_indices=list(np.argsort(scores)[::-1])
        selected_idx=sorted_indices[:2]
        mid_idx=sorted_indices[len(sorted_indices)//2]
        selected_idx.append(mid_idx)
        for idcs in selected_idx:
            melspec=mel_spectrogram(audios[idcs],audio_sr)
            features=melspec.T
            rgb_image=MelspecToImage(features)
            if row.val==0:
                path = f"audio_images/train/{row.primary_label}/{row.filename.split('.')[0]}-{idcs}.png"

            if row.val==1:
                path = f"audio_images/val/{row.primary_label}/{row.filename.split('.')[0]}-{idcs}.png"

            im = Image.fromarray(rgb_image)
            im.save(path)
    elif row.duration < 20:
        melspec=mel_spectrogram(audios[np.argmax(scores)],audio_sr)
        features=melspec.T
        rgb_image=MelspecToImage(features)
        if row.val==0:
            path = f"audio_images/train/{row.primary_label}/{row.filename.split('.')[0]}.png"
        if row.val==1:
            path =f"audio_images/val/{row.primary_label}/{row.filename.split('.')[0]}.png"
            im = Image.fromarray(rgb_image)
            im.save(path)



    


In [24]:
pool = joblib.Parallel(4)
mapper = joblib.delayed(generate_and_save_image)
tasks = [mapper(row) for row in train_meta.itertuples(False) ]
pool(tqdm(tasks))

  0%|          | 0/62874 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,